In [1]:
import sys
import os

# This adds the main project folder ("QUANT-DEV-UNVESTING") to the list of places
# Python looks for code. It goes one level up from the current working directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Purpose
This testing file will be used to test some of the indicators

In [2]:
import yfinance as yf
import pandas as pd
today = pd.Timestamp.now()

start_date_tickers = today - pd.DateOffset(years=1)
end_date = today
tickers = ['EC', 'VOO', 'NU', 'NVDA', 'AAPL']
df = yf.download(tickers, start_date_tickers, end_date, progress=False)['Close']
df

YF.download() has changed argument auto_adjust default to True


Ticker,AAPL,EC,NU,NVDA,VOO
Date,,,,,
2024-06-17,215.661423,11.073669,11.83,130.938644,496.288025
2024-06-18,213.292480,11.289147,11.69,135.537186,497.610291
2024-06-20,208.703949,11.345359,12.08,130.738693,496.228790
2024-06-21,206.524170,11.242305,12.25,126.530029,495.143372
2024-06-24,207.171127,11.870000,11.86,118.072701,493.811188
...,...,...,...,...,...
2025-06-09,201.449997,8.930000,11.92,142.620102,551.250000
2025-06-10,202.669998,9.020000,11.97,143.950012,554.390015
2025-06-11,198.779999,9.310000,12.76,142.830002,552.859985


# Risk Free Rate

In [ ]:
from indicators import AdjustedRiskFreeRate

usa_bond = yf.Ticker('^TNX')
foreign_rfr = usa_bond.history(period='1d')['Close'].iloc[-1] / 100

start_date_exchange = today - pd.DateOffset(years=5)
exchange_data = yf.download(
    tickers="COP=X",
    start=start_date_exchange,
    end=today,
    progress=False,
    multi_level_index=False
)
exchange_returns = exchange_data['Close'].pct_change().dropna()
expected_exchange_rate = exchange_returns.mean() * 252

rfr = AdjustedRiskFreeRate(foreign_rfr, expected_exchange_rate).calculate()
print(f'Adjusted Risk Free Rate: {rfr}')

Adjusted Risk Free Rate: 0.07510917891238034


# Sharpe Ratio

In [8]:
from indicators import SharpeRatio
periods_per_year = 252
for ticker in tickers:
    prices = df[ticker]
    sr = SharpeRatio(prices, rfr, periods_per_year).calculate()
    if sr > 0:
        msg = 'Performance over risk-free rate. ACCEPT.'
    else:
        msg = 'Performance below risk-free rate. REJECT.' 
    print(f'Sharpe for: {ticker} is {sr}. {msg}')
    

Sharpe for: EC is -0.3502703693054734. Performance below risk-free rate. REJECT.
Sharpe for: VOO is 0.2485744093706503. Performance over risk-free rate. ACCEPT.
Sharpe for: NU is 0.11217776290993589. Performance over risk-free rate. ACCEPT.
Sharpe for: NVDA is 0.3096972624570071. Performance over risk-free rate. ACCEPT.
Sharpe for: AAPL is -0.35723688867869313. Performance below risk-free rate. REJECT.
